[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 1, 1, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0]]


In [863]:
class cells:
    def __init__(self,x,y,cellType):
        self.x = x
        self.y = y
        self.cellType = cellType # integer --> should be mapping table 0-->DEAD
        
class fields:
    def __init__(self,matrix):
        self.x_size = len(matrix)
        self.y_size = len(matrix[0])
        tmp = []
        mat = []
        print('initialize empty field {} x {} '.format(len(matrix),len(matrix[0])))
        for i in range(self.x_size):
            tmpx = []
            matx = []
            for j in range(self.y_size):
                tmpx.append(cells(i,j,0))
                matx.append(0)
            tmp.append(tmpx)
            mat.append(matx)
                
        self.field = tmp
        self.matrix = mat
        
    
    def set_field(self,matrix):
        tmp = []
        for i in range(self.x_size):
            tmpx = []
            for j in range(self.y_size):
                tmpx.append(cells(i,j,matrix[i][j]))
            tmp.append(tmpx)
                
        self.field = tmp
        self.matrix = matrix

    def get_matrix(self):  
        return self.matrix
    
    def render(self):
        cellTypeMap={0:'█',1:'A',2: "B",3:'C',4:'D',5:'E'}
        for i in range(self.x_size):
            rowa = ''
            for j in range(self.y_size):
                rowa = rowa+cellTypeMap[self.field[i][j].cellType]
        
            print('{}\r'.format(rowa))
            
    def changeStatus(self):
        
        # depends on geomatry of map..
        # in this case --> sphere
        
        matrix = self.matrix
        new_matrix = []
        for i in range(self.x_size):
            new_row = []
            for j in range(self.y_size):
                # count neighbor cell for 3x3 cell
                cnt = count3x3(i,j,matrix)
                #print(cnt)
                if cnt<2:
                    # die
                    self.field[i][j].cellType=0
                    new_row.append(0)
                elif cnt>3:
                    # die
                    self.field[i][j].cellType=0
                    new_row.append(0)
                else:
                    self.field[i][j].cellType=1
                    new_row.append(1)
                    
            new_matrix.append(new_row)
            self.matrix = new_matrix
            #set_field(new_matrix)
            
            
    def changeStatusV2(self):
        matrix = self.matrix
        new_matrix = []
        for i in range(self.x_size):
            new_row = []
            for j in range(self.y_size):
                #print("i,j : {},{}".format(i,j))
                # state-priority
                # ( can be changed by game setting )
                # 1. breathing
                # 2. rps
                ccnt = count3x3_detail(i,j,matrix,rps)
                cnt = sum(ccnt[1:])
                if cnt<2:
                    # die
                    self.field[i][j].cellType=0
                    new_row.append(0)
                elif cnt>3:
                    # die
                    self.field[i][j].cellType=0
                    new_row.append(0)
                else:
                    # do rps
                    
                    # tie 
                    #   - random ! 
                    #   - no-change
                    # else 
                    #   - rcs > count !
                    #    argmax(ccnt[1:])+1 # since 0 --> dead
                   
                    xa = np.dot(np.array(rps),np.array(ccnt[1:]).transpose())
                    xa_elem = list(np.where(np.array(ccnt[1:])>0)[0])
                    if len(xa_elem)==1:
                        self.field[i][j].cellType = xa_elem[0]+1
                        new_row.append(xa_elem[0]+1)
                    elif np.all(xa[xa_elem]==0):
                        # pure tie...
                        #print('tie')
                        np.where(np.array(ccnt[1:])>0)
                        self.field[i][j].cellType = int(np.random.randint(1,len(xa_elem))+1)
                        new_row.append(int(np.random.randint(1,len(xa_elem)))+1)
                    else:
                        # more priority
                        self.field[i][j].cellType = int(np.argmax(xa))+1
                        new_row.append(int(np.argmax(xa))+1)

            new_matrix.append(new_row)
            self.matrix = new_matrix
            #set_field(new_matrix)

                    
                #rowa = rowa+cellTypeMap[self.field[i][j].cellType]

def count3x3(x,y,matrix):
    summa = 0
    x_size = len(matrix)
    y_size = len(matrix[0])
    for i in range(-1,2):
        for j in range(-1,2):
            if matrix[(x+i)%x_size][(y+j)%y_size]==0:
                plus = 0
            else:
                plus = 1
            #plus = matrix[(x+i)%x_size][(y+j)%y_size]/matrix[(x+i)%x_size][(y+j)%y_size]
            summa += plus

    return summa - matrix[x][y]
        
    
    
def count3x3_detail(x,y,matrix,rps):
    summa = [0]*(len(rps)+1)
    x_size = len(matrix)
    y_size = len(matrix[0])
    for i in range(-1,2):
        for j in range(-1,2):
            #summa += matrix[(x+i)%x_size][(y+j)%y_size]
            summa[matrix[(x+i)%x_size][(y+j)%y_size]] = summa[matrix[(x+i)%x_size][(y+j)%y_size]]+1
    
    summa[matrix[x][y]] = summa[matrix[x][y]]-1 #removing itself
    return summa
            
            

def rpsGenerate(n):
    # make seed [0,-1,1,-1,....]
    # this works 
    # n --> odd
    # n --> even : not accurate
    
    rps = []
    seeda = [0]
    for i in range(1,n):
        seeda.append((-1)**i)
    
    rps.append(seeda)
    A = seeda
    for i in range(1,n):
        A = [A[-1]]+A[:-1]
        rps.append(A)
        
    return rps        
        
def rps_process(x,y,rps=rps):
    # 0 : draw
    # 1 : x wins y
    # -1 : y wins x
    return rps[x][y]



        

In [875]:
#print(matrix[3][4])
import numpy as np

#matrix = [[0,0,0,0,0],[0,0,0,0,0],[0,1,1,3,0],[0,0,2,0,0],[0,0,0,0,0]]

N = 3
W = 10
#p = [0.5,0.1,0.1,0.1,0.1,0.1]
p = [0.7,0.1,0.1,0.1]

rps = rpsGenerate(N)
print(rps)
matrix = [[0,0,0,0,0],[0,0,0,0,0],[0,1,2,3,0],[0,0,0,0,0],[0,0,0,0,0]]
#matrix = [[0,0,0,0,0],[0,0,0,0,0],[0,1,1,1,0],[0,0,1,0,0],[0,0,0,0,0]]
#matrix = np.random.randint(2, size=(100, 100))
#matrix = np.random.choice(N+1,W*W,p=p).reshape(W,W)
print(matrix)
#rps = [[0,-1,1],[1,0,-1],[-1,1,0]]
FF = fields(matrix)
FF.set_field(matrix)

#cnt = count3x3(2,2,matrix)
#ccnt = count3x3_detail(1,3,matrix,rps)



#print(rps)
#np.all()
#print((ccnt[1:]===1).astype(int))


print(set(ccnt[1:]))
print(len(set(ccnt[1:])))
#print(np.argmax(cnt_arr[1:])) # argmax itself cannot catch tie-case
#print('#cells {}'.format(Counter(list(np.array(FF.get_matrix()).reshape(1,100))[0])))
FF.render()

[[0, -1, 1], [1, 0, -1], [-1, 1, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 1, 2, 3, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
initialize empty field 5 x 5 
{0, 1}
2
█████
█████
█ABC█
█████
█████


In [874]:
ccnt = count3x3_detail(1,3,matrix,rps)
cnt = sum(ccnt[1:])
print(ccnt)
print(cnt)

xa = np.dot(np.array(rps),np.array(ccnt[1:]).transpose())
xa_elem = list(np.where(np.array(ccnt[1:])>0)[0])

print(xa,xa_elem)



if len(xa_elem)==1:
    print('single')
    #self.field[i][j].cellType = xa_elem[0]+1
    #new_row.append(xa_elem[0]+1)
elif np.all(xa[xa_elem]==0):
    # pure tie...
    print('tie')
    np.where(np.array(ccnt[1:])>0)
    #self.field[i][j].cellType = int(np.random.randint(1,len(xa_elem))+1)
    #new_row.append(int(np.random.randint(1,len(xa_elem)))+1)
else:
    # more priority
    print('prior')
    print(int(np.argmax(xa))+1)
    #self.field[i][j].cellType = int(np.argmax(xa[xa_elem]))+1
    #new_row.append(int(np.argmax(xa[xa_elem]))+1)

FF.changeStatusV2()
FF.render()

[6, 0, 1, 1]
2
[ 0 -1  1] [1, 2]
prior
3
█████
█████
█████
█████
█████


In [844]:
rps[0][1]

-1

In [876]:
from IPython.display import clear_output
from collections import Counter
import time
for i in range(1000):
    print('epoch {}'.format(i))
    print('#cells {}'.format(Counter(list(np.array(FF.get_matrix()).reshape(1,100))[0])))
    FF.render()
    if np.sum(FF.get_matrix())==0:
        print('no cells')
        break
    else:
        time.sleep(1)
        
    FF.changeStatusV2()
    #FF.changeStatus()
    clear_output()



    

epoch 0


ValueError: cannot reshape array of size 25 into shape (1,100)

In [163]:
np.sum(FF.get_matrix())

0

In [609]:
matrix = [[0,0,0,0,0],[0,0,0,0,0],[0,1,1,3,0],[0,0,2,0,0],[0,0,0,0,0]]


TypeError: unsupported operand type(s) for /: 'list' and 'list'

In [511]:
np.random.randint(1,5)

2